<a href="https://colab.research.google.com/github/kunalavghade/Ai/blob/main/Week_10_%E2%80%94_Backpropagation_Through_Convolution_%26_Pooling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 10 — Backpropagation Through Convolution & Pooling

## Goal

Implement backward propagation for:
- Convolution layer
- ReLU
- Max pooling

Then train a small CNN on MNIST (subset).

By the end of this week, you should:
- Understand convolution gradients mathematically
- Implement conv backward pass
- Implement max-pooling backward pass
- Train CNN end-to-end
- Verify gradients numerically

This is the hardest week of Phase 3.

---

# 1. Why Backprop Through Convolution Is Hard

In fully connected layers:
- Gradients are matrix multiplications.

In convolution:
- Each input pixel affects multiple outputs.
- Filters are reused spatially.
- Gradients must be accumulated carefully.

Backprop becomes structured accumulation.

---

# 2. Forward Recap

For one filter:

Output(i,j) = Σ Σ Input(i+m, j+n) * Kernel(m,n)

Now we need:

∂L/∂Kernel  
∂L/∂Input  

Given ∂L/∂Output.

---

# 3. Gradient w.r.t Kernel

Each kernel weight affects many output locations.

Formula intuition:

dK(m,n) =
    Σ Σ dOutput(i,j) * Input(i+m, j+n)

Meaning:
- Slide input under gradient map.
- Accumulate contributions.

This is cross-correlation with upstream gradient.

---

# 4. Gradient w.r.t Input

Each input pixel contributes to multiple outputs.

Formula intuition:

dInput =
    Convolution(dOutput, rotated_kernel)

Rotate kernel 180° before applying.

---

# 5. Handling Padding in Backward

If forward used padding:
- Backward must remove padding
- Gradients must match original input shape

Always check:
- Shape consistency

---

# 6. ReLU Backward

Simple:

If input > 0 → gradient passes  
Else → gradient = 0

---

# 7. Max Pooling Backward

Key idea:
- Only the max element receives gradient.
- Others get zero.

Store during forward:
- Argmax indices per window

Backward:
- Route gradient to stored location.

---

# 8. Full Backward Flow

For CNN block:

Conv → ReLU → Pool

Backward order:

Pool backward  
→ ReLU backward  
→ Conv backward  

---

# 9. Numerical Gradient Checking

Use finite differences:

∂L/∂θ ≈ (L(θ + ε) − L(θ − ε)) / (2ε)

Compare:
- Analytical gradient
- Numerical gradient

If difference < 1e-6 → correct.

This is mandatory before training.

---

# Coding Exercises

## Question 1: Conv Backward (Single Channel)

Implement:
- dKernel
- dInput

Test:
- Small input (5×5)
- Small kernel (3×3)
- Compare with numerical gradients

---

## Question 2: Multi-Channel Conv Backward

Extend to:
- Multiple filters
- Multiple input channels

Verify shapes carefully.

---

## Question 3: Max Pool Backward

Implement:
- Store argmax during forward
- Route gradient correctly

Test on small example.

---

## Question 4: Build Full CNN Backward

Chain together:
- Conv backward
- ReLU backward
- Pool backward
- FC backward

Test gradient consistency.

---

## Question 5: Train CNN on MNIST (Small Subset)

Steps:
- Load 5k samples
- Train simple CNN
- Use Adam optimizer
- Track loss & accuracy

Goal:
- Reach >90% accuracy on small subset

---

## Question 6: Compare with MLP

Train MLP with similar parameter count.

Compare:
- Accuracy
- Convergence speed
- Generalization

Plot:
- Train loss
- Validation accuracy

---

# Conceptual Questions

1. Why must kernel be rotated for input gradient?
2. Why is gradient accumulation necessary?
3. Why does max pooling backward require stored indices?
4. Why is numerical gradient checking important?
5. Why do CNNs generalize better than MLPs on images?

---

# Outcome of Week 10

After this week, you should:
- Understand convolution backprop deeply
- Trust your gradient implementation
- Train CNN end-to-end
- Appreciate inductive bias mathematically


In [1]:
!pip install tensorflow

In [2]:

import numpy as np
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt

np.random.seed(42)

(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train / 255.0
X_test = X_test / 255.0

X_train = X_train[:5000]
y_train = y_train[:5000]

X_test = X_test[:1000]
y_test = y_test[:1000]

print(X_train.shape)
print(y_train.shape)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
(5000, 28, 28)
(5000,)


**PART 1 — Single Channel Convolution Backward**

We start small:

Input: 5×5

Kernel: 3×3

Stride = 1

No padding

**Question 1 — Conv Backward (Single Channel)**


Forward

In [3]:
def conv_forward(X, K):
    H, W = X.shape
    k = K.shape[0]

    out_h = H - k + 1
    out_w = W - k + 1

    out = np.zeros((out_h, out_w))

    for i in range(out_h):
        for j in range(out_w):
            out[i,j] = np.sum(X[i:i+k, j:j+k] * K)

    cache = (X, K)
    return out, cache


Backward

**dKernel**

𝑑𝐾(𝑚,𝑛)=∑𝑑𝑂𝑢𝑡(𝑖,𝑗)∗𝑋(𝑖+𝑚,𝑗+𝑛)




**dInput**

𝑑𝑋=𝑐𝑜𝑛𝑣(𝑑𝑂𝑢𝑡,𝑟𝑜𝑡𝑎𝑡𝑒𝑑(𝐾))


In [4]:
def conv_backward(dout, cache):
    X, K = cache
    H, W = X.shape
    k = K.shape[0]

    dK = np.zeros_like(K)
    dX = np.zeros_like(X)

    # dKernel
    for m in range(k):
        for n in range(k):
            for i in range(dout.shape[0]):
                for j in range(dout.shape[1]):
                    dK[m,n] += dout[i,j] * X[i+m, j+n]

    # Rotate kernel 180°
    K_rot = np.rot90(K, 2)

    # Pad dout for full convolution
    pad = k - 1
    dout_padded = np.pad(dout, ((pad,pad),(pad,pad)))

    # dInput
    for i in range(H):
        for j in range(W):
            dX[i,j] = np.sum(
                dout_padded[i:i+k, j:j+k] * K_rot
            )

    return dX, dK


🔹 Numerical Gradient Check (MANDATORY)

In [5]:
def numerical_gradient(X, K, eps=1e-5):
    dK_num = np.zeros_like(K)

    for m in range(K.shape[0]):
        for n in range(K.shape[1]):
            K_pos = K.copy()
            K_neg = K.copy()

            K_pos[m,n] += eps
            K_neg[m,n] -= eps

            out_pos, _ = conv_forward(X, K_pos)
            out_neg, _ = conv_forward(X, K_neg)

            loss_pos = np.sum(out_pos)
            loss_neg = np.sum(out_neg)

            dK_num[m,n] = (loss_pos - loss_neg) / (2*eps)

    return dK_num


In [6]:
X = np.random.randn(5,5)
K = np.random.randn(3,3)

out, cache = conv_forward(X,K)
dout = np.ones_like(out)

dX, dK = conv_backward(dout, cache)
dK_num = numerical_gradient(X,K)

print("Difference:", np.linalg.norm(dK - dK_num))


Difference: 2.0921302088569912e-10


**Question 3 — Max Pool Backward**

Forward (store argmax)

In [7]:
def maxpool_forward(X, k = 2):
  H, W = X.shape
  out_h = H // k
  out_w = W // k

  out = np.zeros((out_h, out_w))
  mask = {}

  for i in range(out_h):
    for j in range(out_w):
      window = X[i*k:(i+1)*k, j*k:(j+1)*k]
      max_val = np.max(window)
      out[i, j] = max_val

      idx = np.unravel_index(np.argmax(window), window.shape)
      mask[(i,j)] = (i*k+idx[0], j*k+idx[1])

  cache = (X, mask)
  return out, cache

Backward

In [15]:
def maxpool_backward(dout, cache, k=2):
  X, mask = cache
  dX = np.zeros_like(X)

  for i in range(dout.shape[0]):
    for j in range(dout.shape[1]):
      x_idx, y_idx = mask[(i, j)]
      dX[x_idx, y_idx] = dout[i, j]

  return dX

ReLU Backward

In [17]:
def relu_forward(X):
  out = np.maximum(0, X)
  cache = X
  return out, cache


def relu_backward(dout, cache):
  X = cache
  dX = dout.copy()
  dX[X <= 0] = 0
  return dX

PART 2 — Build CNN With Backward

We’ll build:

Conv → ReLU → Pool → FC → Softmax

FC Layer

In [9]:
class FC:
  def __init__(self, in_dim, out_dim):
    self.W = np.random.randn(out_dim, in_dim) * 0.01
    self.b = np.zeros(out_dim)

  def forward(self, X):
    self.X = X
    return self.W @ self.X + self.b

  def backward(self, dout):
    dW = np.outer(dout, self.X)
    db = dout
    dX = self.W.T @ dout
    return dX, dW, db

Softmax + Cross Entropy

In [12]:
def softmax(x):
  e = np.exp(x - np.max(x))
  return e / np.sum(e)

def cross_entropy(probs, y):
  return -np.log(probs[y])

Full Training Loop (Small CNN)

For simplicity:

1 conv filter

1 pooling

1 FC

In [18]:
K = np.random.randn(3,3)*0.1
fc = FC(13*13,10)

lr = 0.01

for epoch in range(3):
    total_loss = 0

    for i in range(1000):
        X = X_train[i]
        y = y_train[i]

        # Forward
        conv_out, conv_cache = conv_forward(X,K)
        relu_out, relu_cache = relu_forward(conv_out)
        pool_out, pool_cache = maxpool_forward(relu_out)

        flat = pool_out.flatten()
        logits = fc.forward(flat)
        probs = softmax(logits)

        loss = cross_entropy(probs,y)
        total_loss += loss

        # Backward
        dlogits = probs
        dlogits[y] -= 1

        dflat, dW, db = fc.backward(dlogits)

        dpool = dflat.reshape(pool_out.shape)
        drelu = maxpool_backward(dpool, pool_cache)
        dconv = relu_backward(drelu, relu_cache)

        dX, dK = conv_backward(dconv, conv_cache)

        # Update
        K -= lr * dK
        fc.W -= lr * dW
        fc.b -= lr * db

    print("Epoch:", epoch, "Loss:", total_loss/1000)


Epoch: 0 Loss: 1.5987798797476802
Epoch: 1 Loss: 0.5543139232454839
Epoch: 2 Loss: 0.4209488095590292
